In [ ]:
import torch
import numpy as np 
from segment_anything import sam_model_registry, SamPredictor
import os
import matplotlib.pyplot as plt
import cv2
import torch.nn.functional as F

os.environ['CUDA_VISIBLE_DEVICES'] = '7'

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    


def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)



In [ ]:
model_type = "default"
sam_checkpoint = '/disk1/yichen/sam_vit_h_4b8939.pth'
device = 'cuda:0'
image_file = '/disk1/yichen/lerf/debug/sam_feature_1.png'
image = cv2.imread(image_file)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
sam_feature = np.load('/disk1/yichen/lerf/debug/sam_feature_1.npy')
sam_feature = sam_feature.transpose(2,0,1)

# contrastive_feature = np.load('/disk1/yichen/lerf/debug/sam_feature.npy')
# contrastive_feature = contrastive_feature.transpose(2,0,1)

input_point = np.array([[130, 230]])
input_label = np.array([1])
print(sam_feature.shape)
# print(contrastive_feature.shape)

In [ ]:
def similar_func(feature_q, support):
    similar_type = 'l2_exp'
    if similar_type == 'cos':
        feature_q = feature_q / feature_q.norm(dim=-1, keepdim=True)
        support = support / support.norm(dim=-1, keepdim=True)
        pred_1 = F.cosine_similarity(feature_q, support, dim=1)
    elif similar_type == 'l2':
        pred_1 = torch.sum((flatten_feature - support_ray)**2, dim=-1)
        pred_1 = torch.pow(pred_1, 0.5) / flatten_feature.norm(dim=-1, keepdim=False)
    elif similar_type == 'l2_exp':
        pred_1 = torch.exp(-20 * torch.sum((flatten_feature - support_ray)**2, -1))
    return pred_1



with torch.no_grad():
    sam_feature_dim = contrastive_feature.shape
    flatten_feature = contrastive_feature.reshape([sam_feature_dim[0], -1])
    flatten_feature = torch.from_numpy(flatten_feature)
    flatten_feature = flatten_feature.permute(1,0)
    flatten_feature = flatten_feature
    flatten_feature_norm = flatten_feature.norm(dim=-1, keepdim=False)


    support_ray_index = input_point[0][0] + input_point[0][1] * sam_feature_dim[2]
    # support_ray_index = torch.randint(0,  flatten_feature.size(0), (1,))
    support_ray = flatten_feature[support_ray_index]

    # self_support_mask = self_support(flatten_feature, support_ray)
    # self_support_mask = F.cosine_similarity(flatten_feature, support_ray, dim=1)

    self_support_mask = similar_func(flatten_feature, support_ray)
    
    self_support_mask = self_support_mask.reshape(sam_feature_dim[1:])
    self_support_mask = (self_support_mask > 0.005)
    
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(self_support_mask, plt.gca())
    show_points(input_point, input_label, plt.gca())

In [ ]:
def downsample_sam_feature(feature):
    feature_max_dim = np.max(feature.shape[1:])
    padded_feature = np.pad(feature, ((0, 0), (0, feature_max_dim - feature.shape[1]), 
                                      (0, feature_max_dim - feature.shape[2])), mode='constant')
    
    target_res = 64
    factor = feature_max_dim // target_res
    offset = factor // 2 - 1

    downsampled_feature = padded_feature[:, offset::factor, offset::factor]
    print(downsampled_feature.shape)
    return downsampled_feature

In [ ]:
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)


In [ ]:
predictor.set_image(image)

downsampled_feature = downsample_sam_feature(sam_feature)
torch_feature = torch.from_numpy(downsampled_feature).float().to(device)
torch_feature = torch_feature.unsqueeze(0)
predictor.features = torch_feature
# predictor.set_torch_feature(sam_feature)

masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)

for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')